# Elasticsearch Demo

In [1]:
import pandas as pd

hotels = pd.read_pickle('../Storage/Data/hotels.pkl')
reviews = pd.read_pickle('../Storage/Data/preprocessed_reviews.pkl')
sentiments = pd.read_pickle('../Storage/Data/reviews_agg_sentiment.pkl')

hotels.shape, reviews.shape, sentiments.shape

((21420, 19), (21227, 5), (275, 2))

In [2]:
grouped_hotels = hotels[['name', 'reviews.text']].groupby('name').count()
grouped_hotels.index

Index(['A Bed & Breakfast In Cambridge', 'Abbeville Inn', 'Acorn Motor Inn',
       'Adria Motor Inn', 'Ambassadors Inn and Suites',
       'American Star Inn and Suites Atlantic City', 'Americas Best Value Inn',
       'Americas Best Value Inn - Medical Center/airport',
       'Americas Best Value Inn and Suites', 'Americinn Coralville',
       ...
       'Tryp Dusseldorf Krefeld Hotel', 'Tulip Inn Turin West',
       'Una Hotel Forte Dei Marmi', 'Villa Carlotta',
       'Vista Hotel On Lake Tarpon', 'Warwick Denver',
       'Western Inn-glacier Park', 'Wine Valley Lodge',
       'Wingate By Wyndham Pueblo', 'Worldwide Hospitality Supply Company'],
      dtype='object', name='name', length=299)

In [3]:
sentiments.index

Index(['A Bed & Breakfast In Cambridge', 'Acorn Motor Inn',
       'Ambassadors Inn and Suites',
       'American Star Inn and Suites Atlantic City', 'Americas Best Value Inn',
       'Americas Best Value Inn - Medical Center/airport',
       'Americas Best Value Inn and Suites', 'Americinn Coralville',
       'Americinn Lodge & Suites Appleton', 'Amisos Hotel',
       ...
       'Travelodge Ruther Glen', 'Tryp Dusseldorf Krefeld Hotel',
       'Tulip Inn Turin West', 'Una Hotel Forte Dei Marmi', 'Villa Carlotta',
       'Vista Hotel On Lake Tarpon', 'Warwick Denver',
       'Western Inn-glacier Park', 'Wine Valley Lodge',
       'Wingate By Wyndham Pueblo'],
      dtype='object', name='name', length=275)

In [4]:
no_sent_hotel = grouped_hotels.index.difference(sentiments.index, sort=False)
no_sent_hotel

Index(['Abbeville Inn', 'Adria Motor Inn', 'Bailey Hotel', 'Chippewa Hotel',
       'Close For You', 'Club Quarters, Rockefeller Center',
       'Clubhouse At River Country Est', 'Concord Apartments By Boq Lodging',
       'Days Inn Newton', 'Days Inn Tallulah',
       'Holiday Inn Express Hotel & Suites Hillview', 'Hyatt Dulles',
       'La Playa', 'Marriott Springhill Marina', 'Old Wheeler Hotel',
       'Porto Vista Hotel', 'Relax Inn', 'Ritz Hotel', 'Spring Fountain Motel',
       'Springhill Suites By Marriott South Bend/mishawaka', 'Stratford Inn',
       'Studio 6', 'The Dec', 'Worldwide Hospitality Supply Company'],
      dtype='object', name='name')

In [5]:
# look up `no_sent_hotel` in original `hotels` df before preprocessing
no_sent_hotel.isin(hotels.name).sum()

24

In [6]:
# look up `no_sent_hotel` in original `reviews` df after preprocessing
no_sent_hotel.isin(reviews.name).sum()

0

We will notice that there are some hotels dropped out during preprocessing (from 299 it'll be 275, almost 24 hotels), so we won't index them in ElasticSearch beacuse of missing reviews & missing sentiments

In [7]:
grouped_reviews = reviews.groupby('name')
grouped_reviews.get_group('American Star Inn and Suites Atlantic City')

,name,lemmatized,classification,p_pos,p_neg
8200,American Star Inn and Suites Atlantic City,disgusting place stay place horrible bed scum ...,pos,0.646265,0.353735
8201,American Star Inn and Suites Atlantic City,bad motel stay room dirty food ice cream get m...,neg,0.356840,0.643160
8202,American Star Inn and Suites Atlantic City,clean affordable friendly manager desk nice he...,pos,0.734980,0.265020
8203,American Star Inn and Suites Atlantic City,quality room motel low end motel high cost fee...,neg,0.408191,0.591809
8204,American Star Inn and Suites Atlantic City,good value simple hotel fairly clean pleasant ...,pos,0.876419,0.123581
...,...,...,...,...,...
8261,American Star Inn and Suites Atlantic City,descent get room night price good pay,pos,0.747158,0.252842
8262,American Star Inn and Suites Atlantic City,descent staff rude ignorant,pos,0.951923,0.048077
8263,American Star Inn and Suites Atlantic City,descent bathroom bad,neg,0.483656,0.516344
8264,American Star Inn and Suites Atlantic City,bad manner unprofessional motel meet bad manne...,pos,0.866642,0.133358


In [8]:
grouped_reviews.get_group('Americas Best Value Inn').head()

,name,lemmatized,classification,p_pos,p_neg
8890,Americas Best Value Inn,poor adequate chair 1night stand bed good show...,pos,0.615220,0.384780
8891,Americas Best Value Inn,d expect d expect expensive night s stay clean...,pos,0.844569,0.155431
8892,Americas Best Value Inn,surprising good value clean comfortable helpfu...,pos,0.709604,0.290396
8893,Americas Best Value Inn,hotel pretty bad clean lady naked use work cra...,neg,0.000560,0.999440
8894,Americas Best Value Inn,nice turning motel awful,neg,0.128000,0.872000


In [9]:
hotels[hotels['name'] == 'Americas Best Value Inn'].shape

(401, 19)

In [10]:
# group by all hotel cols, get first group
hotels[hotels['name'] == 'Americas Best Value Inn'].groupby([
    'address', 'categories', 'city', 'country', 'latitude', 'longitude',
    'name', 'postalCode', 'province'
]).first(5)


,,,,,,,,,reviews.doRecommend,reviews.id,reviews.rating
address,categories,city,country,latitude,longitude,name,postalCode,province,,,
125 Sloane Garden Rd,Hotels,Boiling Springs,US,35.011314,-81.954798,Americas Best Value Inn,29316,SC,NaN,NaN,2.0
1320 Harrisburg Pike,Hotels,Lancaster,US,40.056960,-76.331290,Americas Best Value Inn,17603,Manor Ridge,NaN,NaN,1.0
2512 W Lincolnway,Hotels,Cheyenne,US,41.119150,-104.849490,Americas Best Value Inn,82001,WY,NaN,NaN,1.0
2731 S Carson St,Hotels,Carson City,US,39.142715,-119.767824,Americas Best Value Inn,89701,Carson Colony,NaN,NaN,5.0
3080 E Colby St,Hotels,Whitehall,US,43.408970,-86.317420,Americas Best Value Inn,49461,MI,NaN,NaN,1.0
740 Broadway St,Hotels,Chico,US,39.726020,-121.837130,Americas Best Value Inn,95928,Chapmantown,NaN,NaN,1.0


Wel will notice that some hotels don't have unique value per each one of those columns: `['address', 'categories', 'city', 'country', 'latitude', 'longitude', 'name', 'postalCode', 'province']`

In [11]:
filtered_hotels = sentiments.index.values
filtered_hotels[:5]

array(['A Bed & Breakfast In Cambridge', 'Acorn Motor Inn',
       'Ambassadors Inn and Suites',
       'American Star Inn and Suites Atlantic City',
       'Americas Best Value Inn'], dtype=object)

## ES Cloud (AWS) Demo

In [2]:
from elasticsearch import Elasticsearch
from ssl import create_default_context
import configparser

# load global configs
config = configparser.ConfigParser()
config.read('../configs.ini')

# load SSL certs.pem
context = create_default_context(cafile=config['ELASTIC']['ssl_cert_path'])

# Found in the 'Manage Deployment' page
CLOUD_ID = config['ELASTIC']['cloud_id']

# Create the client instance
es_client = Elasticsearch(
    cloud_id=CLOUD_ID,
    http_auth=(config['ELASTIC']['username'], config['ELASTIC']['password']),
    scheme=config['ELASTIC']['scheme'],
    port=config['ELASTIC']['port'],
    ssl_context=context,
)

# Successful response!
es_client.info()
# {'name': 'instance-0000000000', 'cluster_name': ...}

{'name': 'instance-0000000001',
 'cluster_name': 'd64f20da8ed348069549b3b387c7eb58',
 'cluster_uuid': 'BGMBBfU9Rtquo6kqDNtG3A',
 'version': {'number': '8.2.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '9905bfb62a3f0b044948376b4f607f70a8a151b4',
  'build_date': '2022-06-08T22:21:36.455508792Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [13]:
# Get Index API
es_client.indices.get(index="*")

{'.ds-logs-enterprise_search.api-default-2022.06.17-000001': {'aliases': {},
  'mappings': {'_meta': {'version': '1.7.0'},
   '_data_stream_timestamp': {'enabled': True},
   'dynamic_templates': [{'match_ip': {'match': 'ip',
      'match_mapping_type': 'string',
      'mapping': {'type': 'ip'}}},
    {'match_message': {'match': 'message',
      'match_mapping_type': 'string',
      'mapping': {'type': 'match_only_text'}}},
    {'strings_as_keyword': {'match_mapping_type': 'string',
      'mapping': {'ignore_above': 1024, 'type': 'keyword'}}}],
   'date_detection': False,
   'properties': {'@timestamp': {'type': 'date'},
    'agent': {'properties': {'build': {'properties': {'original': {'type': 'keyword',
         'ignore_above': 1024}}},
      'ephemeral_id': {'type': 'keyword', 'ignore_above': 1024},
      'hostname': {'type': 'keyword', 'ignore_above': 1024},
      'id': {'type': 'keyword', 'ignore_above': 1024},
      'name': {'type': 'keyword', 'ignore_above': 1024},
      'type': 

In [14]:
# es_client.index(
#     index='lord-of-the-rings',
#     document={
#         'character': 'Aragon',
#         'quote': 'It is not this day.'
#     }
# )

OUTPUT:

    {'_index': 'lord-of-the-rings',
    '_id': 'HR4ydIEB6_mhCbAmtCZD',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 0,
    '_primary_term': 1}


In [15]:
# es_client.index(
#     index='lord-of-the-rings',
#     document={
#         'character': 'Gandalf',
#         'quote': 'A wizard is never late, nor is he early.'
#     }
# )

OUTPUT:

    {'_index': 'lord-of-the-rings',
    '_id': 'Hh4zdIEB6_mhCbAmiibJ',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1} 

In [16]:
# es_client.index(
#     index='lord-of-the-rings',
#     document={
#         'character': 'Frodo Baggins',
#         'quote': 'You are late'
#     }
# )


OUTPUT:

    {'_index': 'lord-of-the-rings',
    '_id': 'Hx4zdIEB6_mhCbAm4Ca1',
    '_version': 1,
    'result': 'created',
    '_shards': {'total': 2, 'successful': 2, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1}

In [17]:
es_client.indices.refresh(index='lord-of-the-rings')

{'_shards': {'total': 2, 'successful': 2, 'failed': 0}}

In [18]:
# filter docs with a query
result = es_client.search(
    index='lord-of-the-rings',
    query={
        'match': {'quote': 'late'}
    }
)

result['hits']['hits']


[{'_index': 'lord-of-the-rings',
  '_id': 'Hx4zdIEB6_mhCbAm4Ca1',
  '_score': 0.5820575,
  '_source': {'character': 'Frodo Baggins', 'quote': 'You are late'}},
 {'_index': 'lord-of-the-rings',
  '_id': 'Hh4zdIEB6_mhCbAmiibJ',
  '_score': 0.37883914,
  '_source': {'character': 'Gandalf',
   'quote': 'A wizard is never late, nor is he early.'}}]

In [19]:
# return all docs
result = es_client.search(
    index='lord-of-the-rings',
    query={
        'match_all': {}
    }
)

result['hits']['hits']


[{'_index': 'lord-of-the-rings',
  '_id': 'HR4ydIEB6_mhCbAmtCZD',
  '_score': 1.0,
  '_source': {'character': 'Aragon', 'quote': 'It is not this day.'}},
 {'_index': 'lord-of-the-rings',
  '_id': 'Hh4zdIEB6_mhCbAmiibJ',
  '_score': 1.0,
  '_source': {'character': 'Gandalf',
   'quote': 'A wizard is never late, nor is he early.'}},
 {'_index': 'lord-of-the-rings',
  '_id': 'Hx4zdIEB6_mhCbAm4Ca1',
  '_score': 1.0,
  '_source': {'character': 'Frodo Baggins', 'quote': 'You are late'}}]

## Convert DataFrame to Python Dictionary

In [20]:
hotel_name = "Arion"

hotel_cols = ['address', 'categories', 'city', 'country',
              'latitude', 'longitude', 'postalCode', 'province']

review_cols = ['reviews.date', 'reviews.dateAdded',
               'reviews.doRecommend', 'reviews.id', 'reviews.rating', 'reviews.text',
               'reviews.title', 'reviews.userCity', 'reviews.username',
               'reviews.userProvince']

review_sentiment_cols = ['lemmatized', 'classification', 'p_pos', 'p_neg']

agg_sentiment_cols = ['p_pos_mean', 'p_neg_mean']


Join them all together after loading preprocessed verions

In [21]:
final_df = hotels.join(reviews.drop(['name'], axis=1), how='inner')[['name'] + hotel_cols + review_cols + review_sentiment_cols] \
    .set_index('name') \
    .join(sentiments, how='inner')

final_df.head(3)


,address,categories,city,country,latitude,longitude,postalCode,province,reviews.date,reviews.dateAdded,...,reviews.title,reviews.userCity,reviews.username,reviews.userProvince,lemmatized,classification,p_pos,p_neg,p_pos_mean,p_neg_mean
name,,,,,,,,,,,,,,,,,,,,,
A Bed & Breakfast In Cambridge,1657 Cambridge St,Hotels,Cambridge,US,42.374972,-71.110408,2138,MA,2014-09-01T00:00:00Z,2016-11-03T22:07:49Z,...,The host is great,NaN,David,NaN,host great breakfast good good thing stay host...,neg,0.007476,0.992524,0.638919,0.361081
A Bed & Breakfast In Cambridge,1657 Cambridge St,Hotels,Cambridge,US,42.374972,-71.110408,2138,MA,2015-07-24T00:00:00Z,2016-11-03T22:07:49Z,...,Bed & Breakfast Stay,NaN,A Traveler,NaN,bed breakfast stay nice close square little ge...,pos,0.832994,0.167006,0.638919,0.361081
A Bed & Breakfast In Cambridge,1657 Cambridge St,Hotels,Cambridge,US,42.374972,-71.110408,2138,MA,2015-10-10T00:00:00Z,2016-11-03T22:07:49Z,...,We will never stay here again.,NaN,A Traveler,NaN,stay poor excuse,neg,0.184476,0.815524,0.638919,0.361081


In [28]:
all_dicts = final_df.reset_index().groupby(['name'] + hotel_cols + agg_sentiment_cols)[review_cols + review_sentiment_cols] \
    .apply(lambda g: g.T.to_json()).reset_index().T.to_json('../Storage/Data/docs.json')


In [39]:
df = pd.read_json('../Storage/Data/docs.json').T
df.head(10)

,name,address,categories,city,country,latitude,longitude,postalCode,province,p_pos_mean,p_neg_mean,0
0,A Bed & Breakfast In Cambridge,1657 Cambridge St,Hotels,Cambridge,US,42.374972,-71.110408,2138,MA,0.638919,0.361081,"{""0"":{""reviews.date"":""2014-09-01T00:00:00Z"",""r..."
1,Acorn Motor Inn,31530 State Route 20,Hotels,Oak Harbor,US,48.288952,-122.657842,98277,WA,0.63088,0.36912,"{""54"":{""reviews.date"":""2016-02-15T00:00:00Z"",""..."
2,Ambassadors Inn and Suites,716 21st St,Hotels,Virginia Beach,US,36.84766,-75.98443,23451,VA,0.592055,0.407945,"{""74"":{""reviews.date"":""2016-03-26T00:00:00Z"",""..."
3,American Star Inn and Suites Atlantic City,232 E White Horse Pike,Hotels,Absecon,US,39.447342,-74.530457,8205,Galloway Township,0.579689,0.420311,"{""178"":{""reviews.date"":""2015-08-01T00:00:00Z"",..."
4,Americas Best Value Inn,125 Sloane Garden Rd,Hotels,Boiling Springs,US,35.011314,-81.954798,29316,SC,0.539331,0.460669,"{""435"":{""reviews.date"":""2015-11-28T00:00:00Z"",..."
5,Americas Best Value Inn,1320 Harrisburg Pike,Hotels,Lancaster,US,40.05696,-76.33129,17603,Manor Ridge,0.539331,0.460669,"{""424"":{""reviews.date"":""2015-10-09T00:00:00Z"",..."
6,Americas Best Value Inn,2512 W Lincolnway,Hotels,Cheyenne,US,41.11915,-104.84949,82001,WY,0.539331,0.460669,"{""244"":{""reviews.date"":""2016-07-13T00:00:00Z"",..."
7,Americas Best Value Inn,2731 S Carson St,Hotels,Carson City,US,39.142715,-119.767824,89701,Carson Colony,0.539331,0.460669,"{""508"":{""reviews.date"":""2015-03-21T00:00:00Z"",..."
8,Americas Best Value Inn,3080 E Colby St,Hotels,Whitehall,US,43.40897,-86.31742,49461,MI,0.539331,0.460669,"{""570"":{""reviews.date"":""2014-08-19T00:00:00Z"",..."
9,Americas Best Value Inn,740 Broadway St,Hotels,Chico,US,39.72602,-121.83713,95928,Chapmantown,0.539331,0.460669,"{""363"":{""reviews.date"":""2016-06-18T00:00:00Z"",..."


In [59]:
df[["name"] + hotel_cols + agg_sentiment_cols].T.to_dict()[0]

{'name': 'A Bed & Breakfast In Cambridge',
 'address': '1657 Cambridge St',
 'categories': 'Hotels',
 'city': 'Cambridge',
 'country': 'US',
 'latitude': 42.374972,
 'longitude': -71.110408,
 'postalCode': '2138',
 'province': 'MA',
 'p_pos_mean': 0.6389192238,
 'p_neg_mean': 0.3610807762}

### Indexing Preprocessed Data In ElasticSearch 

In [66]:
# index all hotels info without reviews
for doc in df[['name'] + hotel_cols + agg_sentiment_cols].T.to_dict().values():
    es_client.index(
        index='demo-hotels-1',
        document=doc
    )

In [6]:
# return docs count
result = es_client.count(
    index='demo-hotels-1',
    # query={
    #     "match_all": {}
    # }
)

result


{'count': 282,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [74]:
# return all docs
result = es_client.search(
    index='demo-hotels-1',
    query={
        "query_string": {
            "query": "NOT (province: MA)"
        }
    }
)

result['hits']['hits']


[{'_index': 'demo-hotels-1',
  '_id': 'eh5ydYEB6_mhCbAmASb_',
  '_score': 0.0,
  '_source': {'name': 'Days Inn and Suites Albany',
   'address': '1606 Central Ave',
   'categories': 'Hotels',
   'city': 'Albany',
   'country': 'US',
   'latitude': 42.718398,
   'longitude': -73.833766,
   'postalCode': '12205',
   'province': 'Colonie Center',
   'p_pos_mean': 0.5186088551,
   'p_neg_mean': 0.4813911449}},
 {'_index': 'demo-hotels-1',
  '_id': 'ex5ydYEB6_mhCbAmAiZG',
  '_score': 0.0,
  '_source': {'name': 'Deluxe Inn Merrillville',
   'address': '732 E 82nd Ave',
   'categories': 'Hotels',
   'city': 'Merrillville',
   'country': 'US',
   'latitude': 41.469446,
   'longitude': -87.329978,
   'postalCode': '46410',
   'province': 'IN',
   'p_pos_mean': 0.2003160672,
   'p_neg_mean': 0.7996839328}},
 {'_index': 'demo-hotels-1',
  '_id': 'fB5ydYEB6_mhCbAmAiaN',
  '_score': 0.0,
  '_source': {'name': 'Discovery Inn',
   'address': '380 W 7200 S',
   'categories': 'Hotels',
   'city': 'Midv

---


### DataFrame MultiColumn Indexing
For more logical representaion

In [ ]:
col_indexes = []
for t in hotel_cols:
    col_indexes.append(('hotel', t))

for t in review_cols + review_sentiment_cols:
    col_indexes.append(('review', t))

for t in agg_sentiment_cols:
    col_indexes.append(('sentiment', t))

pd.MultiIndex.from_tuples(col_indexes)

MultiIndex([(    'hotel',              'address'),
            (    'hotel',           'categories'),
            (    'hotel',                 'city'),
            (    'hotel',              'country'),
            (    'hotel',             'latitude'),
            (    'hotel',            'longitude'),
            (    'hotel',           'postalCode'),
            (    'hotel',             'province'),
            (   'review',         'reviews.date'),
            (   'review',    'reviews.dateAdded'),
            (   'review',  'reviews.doRecommend'),
            (   'review',           'reviews.id'),
            (   'review',       'reviews.rating'),
            (   'review',         'reviews.text'),
            (   'review',        'reviews.title'),
            (   'review',     'reviews.userCity'),
            (   'review',     'reviews.username'),
            (   'review', 'reviews.userProvince'),
            (   'review',           'lemmatized'),
            (   'review',      

In [ ]:
final_df.columns = pd.MultiIndex.from_tuples(col_indexes)
final_df.head()

hotel                        \
                                          address categories       city   
name                                                                      
A Bed & Breakfast In Cambridge  1657 Cambridge St     Hotels  Cambridge   
A Bed & Breakfast In Cambridge  1657 Cambridge St     Hotels  Cambridge   
A Bed & Breakfast In Cambridge  1657 Cambridge St     Hotels  Cambridge   
A Bed & Breakfast In Cambridge  1657 Cambridge St     Hotels  Cambridge   
A Bed & Breakfast In Cambridge  1657 Cambridge St     Hotels  Cambridge   

                                                                         \
                               country   latitude  longitude postalCode   
name                                                                      
A Bed & Breakfast In Cambridge      US  42.374972 -71.110408       2138   
A Bed & Breakfast In Cambridge      US  42.374972 -71.110408       2138   
A Bed & Breakfast In Cambridge      US  42.374972 -71.110408       2138   
A Bed & Breakfast In Cambridge      US  42.374972 -71.110408       2138   
A Bed & Breakfast In Cambridge      US  42.374972 -71.110408       2138   

                                                       review  \
                               province          reviews.date   
name                                                            
A Bed & Breakfast In Cambridge       MA  2014-09-01T00:00:00Z   
A Bed & Breakfast In Cambridge       MA  2015-07-24T00:00:00Z   
A Bed & Breakfast In Cambridge       MA  2015-10-10T00:00:00Z   
A Bed & Breakfast In Cambridge       MA  2014-06-17T00:00:00Z   
A Bed & Breakfast In Cambridge       MA  2014-10-23T00:00:00Z   

                                                      ...  \
                                   reviews.dateAdded  ...   
name                                                  ...   
A Bed & Breakfast In Cambridge  2016-11-03T22:07:49Z  ...   
A Bed & Breakfast In Cambridge  2016-11-03T22:07:49Z  ...   
A Bed & Breakfast In Cambridge  2016-11-03T22:07:49Z  ...   
A Bed & Breakfast In Cambridge  2016-11-03T22:07:49Z  ...   
A Bed & Breakfast In Cambridge  2016-11-03T22:07:49Z  ...   

                                                                \
                                                 reviews.title   
name                                                             
A Bed & Breakfast In Cambridge               The host is great   
A Bed & Breakfast In Cambridge            Bed & Breakfast Stay   
A Bed & Breakfast In Cambridge  We will never stay here again.   
A Bed & Breakfast In Cambridge                      Lovely B&B   
A Bed & Breakfast In Cambridge          Misleading advertising   

                                                                  \
                               reviews.userCity reviews.username   
name                                                               
A Bed & Breakfast In Cambridge              NaN            David   
A Bed & Breakfast In Cambridge              NaN       A Traveler   
A Bed & Breakfast In Cambridge              NaN       A Traveler   
A Bed & Breakfast In Cambridge              NaN             Sara   
A Bed & Breakfast In Cambridge              NaN       A Traveler   

                                                     \
                               reviews.userProvince   
name                                                  
A Bed & Breakfast In Cambridge                  NaN   
A Bed & Breakfast In Cambridge                  NaN   
A Bed & Breakfast In Cambridge                  NaN   
A Bed & Breakfast In Cambridge                  NaN   
A Bed & Breakfast In Cambridge                  NaN   

                                                                                   \
                                                                       lemmatized   
name                                                                                
A Bed & Breakfast In Cambridge  host great breakfast good good

In [ ]:
# import uuid

# def doc_generator(df):
#     df_iter = df.iterrows()
#     for index, document in df_iter:
#         yield {
#                 "_index": 'es_demo',
#                 "_doc_type": "_doc",
#                 "_id" : uuid.uuid4(),
#                 "_source": document,
#             }
            

# helpers.bulk(es_client, doc_generator(sentiments))